In [ ]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np

from librosa.util import buf_to_float
from librosa.display import waveplot, specshow
from librosa import amplitude_to_db, stft, power_to_db

In [ ]:
fft_size = 512
sample_rate = 16000
n_mels = 64

In [ ]:
def play_wav(f_name):
    sr, wav_data = wavfile.read(f_name)
    ipd.display(ipd.Audio(f_name, rate=sr))
    
    return wav_data
    
def show_wav(f_name):
    sr, wav_data = wavfile.read(f_name)
    wav_data = buf_to_float(wav_data)
    plt.figure(figsize=(14, 4))
    plt.grid()
    waveplot(wav_data, sr=sr)
    
    return wav_data
    
def show_spec(f_name):
    sr, wav_data = wavfile.read(f_name)
    wav_data = buf_to_float(wav_data)
    spec = stft(wav_data, n_fft=fft_size)
    spec_db = amplitude_to_db(abs(spec))
    plt.figure(figsize=(14, 8))
    specshow(spec_db, sr=sr, x_axis='time', y_axis='hz')
    
    return spec

Обратное преобразование Фурье (представляем сигнал в виде суммы гармоник): $$ x_n = {{1}\over{N}}\sum\limits_{k=0}^{N-1} X_k e^{{{2\pi i}\over{N}} k n} $$

Прямое преобразование Фурье: $$ X_k = \sum\limits_{n=0}^{N-1} x_n e^{-{{2\pi i}\over{N}} k n} $$

Оконное преобразование Фурье: $$ F(m, k) = \sum\limits_{n=-\infty}^{+\infty} x[n] w[n-m] e^{-i k n} $$

In [ ]:
f_name = './spec_expl.wav'

play_wav(f_name)
show_wav(f_name)
spec = show_spec(f_name)

In [ ]:
from librosa.core import fft_frequencies
from librosa.filters import mel

from sklearn.preprocessing import MinMaxScaler

Переход к мел шкале от частоты
$$ mel = 2595 \ln(1 + {{freq}\over{700}}) $$

In [ ]:
def freq_to_mel(f):
    return 2595.0 * np.log10(1.0 + f / 700.0)

freq = np.arange(0, 8001)
mels = freq_to_mel(freq)
plt.figure(figsize=(8, 4))
plt.plot(freq, mels, label='мел(Гц)')
plt.plot(freq, freq, label='y=x')
plt.grid()
plt.legend()
plt.xlabel('Частота, Гц')
plt.ylabel('Мел')
plt.show()

In [ ]:
n_mels_demo = 10
# ---------------------
# Код, выдернутый из функции mel (https://librosa.github.io/librosa/_modules/librosa/filters.html#mel)
# для визуализации линейного размещения мел фильтров на мел оси
max_mel = freq_to_mel(sample_rate//2)
mel_f = np.linspace(0, sample_rate//2, n_mels_demo + 2)

weights = np.zeros((n_mels_demo, int(1 + fft_size // 2)))

fftfreqs = fft_frequencies(sr=sample_rate, n_fft=fft_size)
fdiff = np.diff(mel_f)
ramps = np.subtract.outer(mel_f, fftfreqs)

for i in range(n_mels_demo):
    # lower and upper slopes for all bins
    lower = -ramps[i] / fdiff[i]
    upper = ramps[i+2] / fdiff[i+1]

    # .. then intersect them with each other and zero
    weights[i] = np.maximum(0, np.minimum(lower, upper))
# ---------------------

# Рисуем полученные фильтры на мел шкале
plt.figure(figsize=(14, 4))
plt.title('{} фильтров на мел-шкале'.format(n_mels_demo))
for f in weights:
    plt.plot(f)
plt.xlabel('Мел')
plt.grid()
freq_labels = [freq for freq in range(0, int(max_mel), 500)]
plt.xticks(ticks=np.linspace(0, fft_size//2 + 1, len(freq_labels)), labels=freq_labels)
plt.show()

# Рисуем полученные фильтры на частотной шкале
mel_matrix = mel(sr=sample_rate, n_fft=fft_size, n_mels=n_mels_demo)
# делаем фильтры "высотой" единица
mel_matrix = MinMaxScaler().fit_transform(mel_matrix.T).T

plt.figure(figsize=(14, 4))
plt.title('{} фильтров на частотной шкале'.format(n_mels_demo))
for f in mel_matrix:
    plt.plot(f)
plt.grid()
plt.xlabel('Частота, Гц')
freq_labels = [freq for freq in range(0, sample_rate//2 + 1, 1000)]
plt.xticks(ticks=np.linspace(0, fft_size//2 + 1, len(freq_labels)), labels=freq_labels)
plt.show()

In [ ]:
# теперь возьмем побольше фильтров, нарисуем один из первых на спектре речевого фрагмента нашей вавки
mel_matrix = mel(sr=sample_rate, n_fft=fft_size, n_mels=n_mels)
mel_matrix_ = MinMaxScaler().fit_transform(mel_matrix.T).T

# Рисуем полученные фильтры на частотной шкале
plt.figure(figsize=(14, 4))
plt.title('{} фильтра на мел-шкале'.format(n_mels))
for f in mel_matrix_:
    plt.plot(f)
plt.grid()
plt.xlabel('Частота, Гц')
freq_labels = [freq for freq in range(0, sample_rate//2 + 1, 1000)]
plt.xticks(ticks=np.linspace(0, fft_size//2 + 1, len(freq_labels)), labels=freq_labels)
plt.show()

# отдельный фильтр вместе со спектром
second_filter = mel_matrix_[2]
sixtyth_filter = mel_matrix_[59]
spec_part = np.log10(np.abs(spec[:, 100]))
spec_part -= spec_part.mean()

plt.figure(figsize=(14, 4))
plt.plot(spec_part, label='Спектр')
plt.title('3-ий и 60-ый фильтр, изображенный на речевом спектре')
plt.plot(second_filter*2, label='Мел-фильтр 3')
plt.plot(sixtyth_filter*2, label='Мел-фильтр 60')
plt.legend()
plt.grid()
plt.xlabel('Частота, Гц')
freq_labels = [freq for freq in range(0, sample_rate//2 + 1, 1000)]
plt.xticks(ticks=np.linspace(0, fft_size//2 + 1, len(freq_labels)), labels=freq_labels)

plt.show()

In [ ]:
print('Форма матрицы фильтров:', mel_matrix.shape)
print('Форма спектрограммы:', spec.shape)

mel_spec = np.dot(mel_matrix, spec)

print('Форма мел-спектрограммы:', mel_spec.shape)

In [ ]:
spec_db = amplitude_to_db(abs(spec))
plt.figure(figsize=(14, 4))
specshow(spec_db, sr=sample_rate, x_axis='time', y_axis='hz')
plt.title('Спектрограмма')
plt.show()

mel_spec_db = amplitude_to_db(abs(mel_spec))
plt.figure(figsize=(14, 4))
plt.title('Мел-спектрограмма')
specshow(mel_spec_db, sr=sample_rate, x_axis='time', y_axis='mel')
plt.show()


# SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET

https://github.com/mravanelli/SincNet/

Одномерная свертка: $$ y[n] = x[n] * h[n] = \sum\limits_{k=0}^{L-1}{x[k]h[n-k]}  $$


Главная идея в параметрической свертке: $$ y[n] = x[n] * g[n, \theta] $$

Представим, что с помощью этой свертки мы хотим обратить внимание сети на какую-то конкретную полосу частот.
Добиться этого можно, используя прямогольные фильтры в частотной области.
$$ G(f, f_1, f_2) = rect({{f}\over{2 f_2}}) - rect({{f}\over{2 f_1}})  $$

In [ ]:
def rect(w):
    _w = np.abs(w)
    res = np.ones(len(w)).astype(float)
    res[_w > 0.5] = 0.
    res[_w == 0.5] = 0.5

    return res

def G(f, f1, f2):
    return rect(0.5*f/f2) - rect(0.5*f/f1)

f1 = 100
f2 = 120

w = np.linspace(0, 350, 350)
plt.plot(w, G(w, f1, f2))
plt.grid()
plt.xlabel('Частота, Гц')
plt.show()

Чтобы перейти во временную область для данной фукнции, надо посчитать обратное преобразование Фурье для прямоугольной функции.
Оно рассчитывается аналитически и будет равно следующему выражению.


$$ g(n, f_1, f_2) = 2 f_2 sinc(2 \pi f_2 n) - 2 f_1 sinc(2 \pi f_1 n)$$

$$ sinc(t) = {{\sin(t)}\over{t}}$$


In [ ]:
def sinc(t):
    return np.sin(t)/t

def g(t, f1, f2):
    return 2*f2*sinc(2*np.pi*f2*t) - 2*f1*sinc(2*np.pi*f1*t)
t0 = 0.1

t = np.linspace(-t0, t0, 300)
plt.plot(t, g(t, f1, f2))
plt.grid()
plt.show()